In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses, optimizers, regularizers
from tensorflow.keras.models import Model

2025-04-04 16:01:50.337704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743775310.350123  143908 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743775310.354182  143908 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743775310.363691  143908 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743775310.363700  143908 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743775310.363702  143908 computation_placer.cc:177] computation placer alr

In [2]:
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1.0
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0


In [3]:
# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

In [4]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

I0000 00:00:1743775314.102696  143908 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5


In [5]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [51]:
def model_builder(hp):

    hp_units_layer_1 = hp.Int('units_layer_1', min_value=8, max_value=64, step=8)
    hp_units_layer_2 = hp.Int('units_layer_2', min_value=8, max_value=64, step=8)
    hp_units_layer_3 = hp.Int('units_layer_3', min_value=8, max_value=64, step=8)
    hp_units_layer_4 = hp.Int('units_layer_4', min_value=8, max_value=64, step=8)
    
    hp_activation_layer_1 = hp.Choice("activation_layer_1", values=["relu","sigmoid"])
    hp_activation_layer_2 = hp.Choice("activation_layer_2", values=["relu","sigmoid"])
    hp_activation_layer_3 = hp.Choice("activation_layer_3", values=["relu","sigmoid"])
    hp_activation_layer_4 = hp.Choice("activation_layer_4", values=["relu","sigmoid"])

    hp_regulizer_layer_1 = hp.Choice("regulizer_layer_1", values=["L1","L2"])
    hp_regulizer_layer_2 = hp.Choice("regulizer_layer_2", values=["L1","L2"])
    hp_regulizer_layer_3 = hp.Choice("regulizer_layer_3", values=["L1","L2"])
    hp_regulizer_layer_4 = hp.Choice("regulizer_layer_4", values=["L1","L2"])
    
    model = tf.keras.Sequential([
        layers.Input(shape=(140,)),        
        layers.Dense(units=hp_units_layer_1, activation=hp_activation_layer_1, kernel_regularizer=hp_regulizer_layer_1),
        layers.Dense(units=hp_units_layer_2, activation=hp_activation_layer_2, kernel_regularizer=hp_regulizer_layer_2),
        layers.Dense(8, activation="relu"),
        layers.Dense(units=hp_units_layer_3, activation=hp_activation_layer_3, kernel_regularizer=hp_regulizer_layer_3),
        layers.Dense(units=hp_units_layer_4, activation=hp_activation_layer_4, kernel_regularizer=hp_regulizer_layer_4),
        layers.Dense(140, activation="sigmoid")
    ])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss=losses.MeanAbsoluteError(), 
                  metrics=["mae"])
    return model

In [ ]:
%%time
import shutil

shutil.rmtree('my_dir', ignore_errors=True)

tuner = kt.Hyperband(
    model_builder,
    objective='mae',
    max_epochs=40,
    directory='my_dir',
    project_name='ECG-EKG'
)

tuner.search(normal_train_data, normal_train_data, 
             epochs=20, 
             batch_size=512, 
             validation_data=(test_data, test_data), 
             shuffle=True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 54 Complete [00h 08m 05s]
mae: 0.05356455221772194

Best mae So Far: 0.020532282069325447
Total elapsed time: 00h 15m 02s

Search: Running Trial #55

Value             |Best Value So Far |Hyperparameter
64                |8                 |units_layer_1
16                |40                |units_layer_2
56                |16                |units_layer_3
48                |24                |units_layer_4
relu              |sigmoid           |activation_layer_1
sigmoid           |sigmoid           |activation_layer_2
sigmoid           |relu              |activation_layer_3
sigmoid           |relu              |activation_layer_4
L1                |L2                |regulizer_layer_1
L1                |L1                |regulizer_layer_2
L1                |L2                |regulizer_layer_3
L1                |L2                |regulizer_layer_4
0.001             |0.01              |learning_rate
5                 |40                |tuner/epochs
0                 |14       

In [46]:
%%time
hypermodel = tuner.hypermodel.build(best_hps)

history = hypermodel.fit(normal_train_data, normal_train_data,
        epochs=20,
        batch_size=512,
        validation_data=(test_data, test_data),
        shuffle=True,
        verbose=0)

CPU times: user 3.71 s, sys: 460 ms, total: 4.17 s
Wall time: 4.15 s


In [47]:
def predict(model, data, threshold):
    reconstructions = model(data)
    loss = tf.keras.losses.mae(reconstructions, data)
    return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
    print("Accuracy = {}".format(accuracy_score(labels, predictions)))
    print("Precision = {}".format(precision_score(labels, predictions)))
    print("Recall = {}".format(recall_score(labels, predictions)))

In [48]:
reconstructions = hypermodel.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [49]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  0.03295753


In [50]:
preds = predict(hypermodel, test_data, threshold)
print_stats(preds, test_labels)

Accuracy = 0.944
Precision = 0.9921875
Recall = 0.9071428571428571
